In [3]:
import os
import glob
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define markers specifically for invasion
MMR_markers = ["MLH1", "MSH2", "PMS2", "MSH6"]

# Define directory for .h5ad files
directory = '/Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples'
file_paths = glob.glob(os.path.join(directory, '*.h5ad'))

# Loop over each file path
for file_path in file_paths:
    # Read the adata file
    adata = sc.read_h5ad(file_path)

    # Extract histological image
    library_id = list(adata.uns['spatial'].keys())[0]  # Use the first library
    image_data = adata.uns['spatial'][library_id]['images']['hires']  # High-res image
    scale_factor = adata.uns['spatial'][library_id]['scalefactors']['tissue_hires_scalef']  # Scale factor for spatial coords

    # Initialize a DataFrame to store the sum of normalized expression values for invasion markers per cell
    cells_marker_expression = pd.DataFrame(0, index=adata.obs.index, columns=['MMR Score'])

    # Calculate the normalized invasion score
    for gene in MMR_markers:
        if gene in adata.var_names:  # Check if the gene is present
            gene_expression = adata[:, gene].X.toarray().ravel()
            # Normalize the expression (Z-score)
            if np.std(gene_expression) != 0:
                normalized_expression = (gene_expression - np.mean(gene_expression)) / np.std(gene_expression)
                cells_marker_expression['MMR Score'] += normalized_expression
            else:
                print(f"Gene {gene} has zero variance in {os.path.basename(file_path)}. Skipping normalization...")
        else:
            print(f"Gene {gene} not found in dataset. Skipping...")

    # Skip if no markers contribute to the score
    if cells_marker_expression['MMR Score'].sum() == 0:
        print(f"No MMR markers found for {os.path.basename(file_path)}. Skipping file...")
        continue

    # Scale spatial coordinates to match the image
    spatial_coords = adata.obsm['spatial'] * scale_factor

    # Plotting
    plt.figure(figsize=(10, 8))
    plt.imshow(image_data, origin='lower')  # Plot histological image

    # Overlay heatmap with adjusted transparency and dot size
    scatter = plt.scatter(
        spatial_coords[:, 0], 
        spatial_coords[:, 1], 
        c=cells_marker_expression['MMR Score'],
        cmap='plasma',  # Color map for intensity
        alpha=0.3,       # Increase transparency
        s=7              # Decrease dot size
    )

    # Add color bar and labels
    plt.colorbar(scatter, label='Normalized MMR Score', shrink=0.8)
    plt.axis('off')  # Turn off axes for cleaner visualization
    plt.title(f'Spatial Distribution of Normalized MMR Markers\n{os.path.basename(file_path)}', fontsize=14)

    # Save the plot
    output_path = f"{os.path.splitext(file_path)[0]}_overlay_MMR_markers.png"
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()
    print(f"Saved plot to {output_path}")

Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/51_A10-38_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/128_B9-102_B12_1_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/80_A12-96_A8_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/092534_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/83-3_1_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/100-52_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/40_B10-18_G4_0_2_adata_overlay

/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/091759_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/0_B11-10_A5_1_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/91_A4-14_A4_1_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/092842_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/36_B19-49_A13_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/11_C9-30_A6_0_adata_overlay_MMR_markers.png
Saved plot to /Users/adiallo/Desktop/Thesis/Data_Documents/All_Data/Visium_samples/Visium_Data/samples/40_B10-18_G4_0_adata_ov